In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
import matplotlib.pyplot as plt


import numpy as np
import wandb
import pandas as pd
import cv2
from tqdm import tqdm
import argparse
from PIL import Image

from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [24]:
class Model_EfficientNet(nn.Module):
    def __init__(self, model_type):
        super(Model_EfficientNet, self).__init__()

        if model_type == "EfficientNet_v2_s":
            self.model = models.efficientnet.efficientnet_v2_s(pretrained=True)
        elif model_type == "EfficientNet_v2_m":
            self.model = models.efficientnet.efficientnet_v2_m(pretrained=True)
        elif model_type == "EfficientNet_v2_l":
            self.model = models.efficientnet.efficientnet_v2_l(pretrained=True)
        elif model_type == "EfficientNet_B0":
            self.model = models.efficientnet.efficientnet_b0(pretrained=True)
        elif model_type == "EfficientNet_B1":
            self.model = models.efficientnet.efficientnet_b1(pretrained=True)
        elif model_type == "EfficientNet_B2":
            self.model = models.efficientnet.efficientnet_b2(pretrained=True)
        elif model_type == "EfficientNet_B3":
            self.model = models.efficientnet.efficientnet_b3(pretrained=True)
        elif model_type == "EfficientNet_B4":
            self.model = models.efficientnet.efficientnet_b4(pretrained=True)
        elif model_type == "EfficientNet_B5":
            self.model = models.efficientnet.efficientnet_b5(pretrained=True)
        elif model_type == "EfficientNet_B6":
            self.model = models.efficientnet.efficientnet_b6(pretrained=True)
        elif model_type == "EfficientNet_B7":
            self.model = models.efficientnet.efficientnet_b7(pretrained=True)
        self.model.classifier[1] = nn.Linear(2560, 10, bias=True)
    def forward(self, x):
        x = self.model(x)
        return x

In [25]:
class Digit_Dataset(Dataset):
    def __init__(self, df_data, transforms=None):
        self.data = df_data
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data.iloc[idx, 1:].values.reshape(28, 28).astype(np.uint8)
        # 3 channels
        img = np.stack((img,) * 3, axis=-1)
        label = self.data.iloc[idx, 0]
        if self.transforms:
            img = self.transforms(img)
        return img, label


In [26]:
class Training_Model:
    def __init__(self, model_type, model, optimizer, criterion, dict_dataloader):
        self.model_type = model_type
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_loader = dict_dataloader["train_loader"]
        self.val_loader = dict_dataloader["val_loader"]
        self.best_f1_score = 0

    def train(self, epochs):
        print("============TRAINING START {}============".format(epochs))
        self.model.train()
        total_loss = 0
        preds, targets = [], []
        for batch_idx, (data, target) in enumerate(tqdm(self.train_loader)):
            data = data.to(device)
            target = target.to(device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            preds += predicted.tolist()
            targets += target.tolist()

        total_loss = round(total_loss / len(self.train_loader), 4)
        precision_scr = round(
            precision_score(targets, preds, average="macro"),
            4,
        )
        accuracy_scr = round(
            accuracy_score(targets, preds),
            4,
        )
        train_f1_scr = round(
            f1_score(targets, preds, average="macro"),
            4,
        )
        print(
            "train_loss: {}, train_precision: {}, train_accuracy: {}, train_f1_score: {}".format(
                total_loss, precision_scr, accuracy_scr, train_f1_scr
            )
        )
        wandb.log(
            {
                "train_loss": total_loss,
                "train_precision_score": precision_scr,
                "train_accuracy_score": accuracy_scr,
                "train_f1_score": train_f1_scr,
            }
        )

    def validation(self, epochs):
        print("============VAL START {}============".format(epochs))
        self.model.eval()
        total_loss = 0
        preds, targets = [], []
        for batch_idx, (data, target) in enumerate(tqdm(self.val_loader)):
            data = data.to(device)
            target = target.to(device)

            with torch.no_grad():
                output = self.model(data)
                loss = self.criterion(output, target)
                total_loss += loss.item()
                _, predicted = torch.max(output.data, 1)

                preds += predicted.tolist()
                targets += target.tolist()
        total_loss = round(total_loss / len(self.val_loader), 4)
        precision_scr = round(precision_score(targets, preds, average="macro"), 4)
        accuracy_scr = round(accuracy_score(targets, preds), 4)
        val_f1_scr = round(f1_score(targets, preds, average="macro"), 4)
        print(
            "val_loss: {}, val_precision: {}, val_accuracy: {}, val_f1_score: {}".format(
                total_loss, precision_scr, accuracy_scr, val_f1_scr
            )
        )
        wandb.log(
            {
                "val_loss": total_loss,
                "val_precision_score": precision_scr,
                "val_accuracy_score": accuracy_scr,
                "val_f1_score": val_f1_scr,
            }
        )
        if val_f1_scr > self.best_f1_score and val_f1_scr > 0.99   :
            self.best_f1_score = val_f1_scr
            torch.save(
                self.model.state_dict(),
                "/kaggle/working/{}_f1_{}.pt".format(self.model_type, val_f1_scr),
            )
#             torch.jit.save(
#                 torch.jit.script(self.model),
#                 self.model.state_dict(),
#                 "/kaggle/working/{}_jit_f1_{}.pt".format("vgg_16", val_f1_scr),
#             )

In [27]:
def get_loader(csv_file, transforms, batch_size=64):
    dataset = Digit_Dataset(csv_file, transforms)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader


def get_input():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_type", type=str, default="EfficientNet_v2_s")
    parser.add_argument("--batch_size", type=int, default=8)
    parser.add_argument("--epochs", type=int, default=20)
    parser.add_argument("--lr", type=float, default=0.001)
    parser.add_argument("--seed", type=int, default=10)
    parser.add_argument("--weight_decay", type=float, default=0.0001)
    args = parser.parse_args()
    return args


def config_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [28]:
config_seed(10)

batch_size = 128
epochs = 50
lr = 0.001
weight_decay = 0.0001

data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

train_data, val_data = train_test_split(data, test_size=0.2, random_state=10)

transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((33, 33)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)

train_loader = get_loader(train_data, transforms=transforms, batch_size=batch_size)
val_loader = get_loader(val_data, transforms=transforms, batch_size=batch_size)

dict_dataloader = {"train_loader": train_loader, "val_loader": val_loader}
model_type = "EfficientNet_B7"
model = Model_EfficientNet(model_type)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

wandb.init(
    project="Digit_Recognizer",
    group="EfficientNet",
    name="{}_{}_{}_{}".format(model_type, batch_size, epochs, lr),
)

wandb.watch(model)

training_model = Training_Model(
    model_type=model_type,
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    dict_dataloader=dict_dataloader
)

for epoch in range(epochs):
    training_model.train(epoch)
    training_model.validation(epoch)

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth


  0%|          | 0.00/255M [00:00<?, ?B/s]

============TRAINING START 0============


100%|██████████| 263/263 [02:07<00:00,  2.07it/s]


train_loss: 0.175, train_precision: 0.9527, train_accuracy: 0.953, train_f1_score: 0.9526
============VAL START 0============


...

val_loss: 0.0304, val_precision: 0.9915, val_accuracy: 0.9914, val_f1_score: 0.9914
============TRAINING START 49============


100%|██████████| 263/263 [01:58<00:00,  2.21it/s]


train_loss: 0.007, train_precision: 0.9983, train_accuracy: 0.9983, train_f1_score: 0.9983
============VAL START 49============


100%|██████████| 66/66 [00:12<00:00,  5.46it/s]

val_loss: 0.0337, val_precision: 0.9904, val_accuracy: 0.9904, val_f1_score: 0.9903


In [29]:
model_type = "EfficientNet_B7"
model = Model_EfficientNet(model_type)

In [30]:
model.load_state_dict(torch.load("/kaggle/working/EfficientNet_B7_f1_0.9923.pt"))

<All keys matched successfully>

In [31]:
model.eval()

Model_EfficientNet(
  (model): EfficientNet(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
              (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (sc

In [32]:
model = model.to(device)

In [40]:
DF_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
DF_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
from torchvision import transforms
DF_submission = pd.DataFrame(columns=["ImageId", "Label"])
transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((33, 33)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)
for row in tqdm(range(DF_test.shape[0])):
    img = DF_test.iloc[row, :].values
    img = img.reshape(28, 28).astype(np.uint8)
#     plt.imshow(img)
#     plt.show()
    # 3 channel
    img = np.stack((img,) * 3, axis=-1)
    img = transforms(img)
    
    img = img.to(device)
    img = img.unsqueeze(0)
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output.data, 1)
        DF_test.iloc[row, :] = predicted.item()
#         print(predicted.item())
        DF_submission = DF_submission.append( {"ImageId": row+1, "Label": predicted.item()}, ignore_index=True)
DF_submission.to_csv("/kaggle/working/EfficientNet_B7_f1_0.9923_submission.csv", index=False)

100%|██████████| 28000/28000 [14:17<00:00, 32.64it/s]


In [42]:
# !rm -rf /kaggle/working/EfficientNet_B7_f1_0.9923_submission.csv